In [1]:
%load_ext lab_black

In [10]:
import pandas as pd

from mppsteel.config.model_scenarios import DEFAULT_SCENARIO
from mppsteel.config.model_config import USD_TO_EUR_CONVERSION_DEFAULT, PROJECT_PATH

from mppsteel.model_solver.solver import choose_technology

In [7]:
scenario_dict = DEFAULT_SCENARIO.copy()
scenario_dict["usd_to_eur"] = USD_TO_EUR_CONVERSION_DEFAULT
scenario_dict["eur_to_usd"] = 1.0 / scenario_dict["usd_to_eur"]

In [ ]:
class ChooseTechnologyInput:
    @classmethod
    def from_filesystem(cls, project_dir=PROJECT_PATH):
        

In [8]:
choose_technology(scenario_dict)

2022-06-02 11:02:23,410 — mppsteel.model_solver.solver — INFO — Creating Steel plant df


FileNotFoundError: [Errno 2] No such file or directory: 'mppsteel/data/pkl_data/formatted_import_data/steel_plants_processed.pickle'